In [ ]:
import os
import arcpy
import pandas as pd
import numpy as np
import csv
from arcgis.features import GeoAccessor, GeoSeriesAccessor

arcpy.env.overwriteOutput = True

<h1>RELACIONES SOBRE LA LOCAL</h1>

<h4>Creación de relaciones M:N</h4>

In [ ]:
arcpy.env.workspace = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.gdb"

feature_municipio = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.gdb\municipio"
tabla_capa = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.gdb\tbl_capa"
tabla_cuenca = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.gdb\cuenca"

# ********************** Relación Municipio - Capa **********************
# - Creación de la estrucutura de la relación
r_municipio_capa = "r_municipio_capa"
arcpy.management.CreateRelationshipClass(tabla_capa, feature_municipio, r_municipio_capa, 'SIMPLE', 'capa', 'municipio', 
                                         "NONE", 'MANY_TO_MANY', "NONE", 'codigo_capa', 'codigo_capa', 'codigo_municipio', 'codigo_municipio')
print("Creada relación: {0}".format(r_municipio_capa))

# - Creación de la vista de tabla
ruta_csv_relacion_municipio_capa = r"D:\PUBLIC\PGAR\Resultados\2.Py\r_municipio_capa.csv"

# - Unificación de registros
arcpy.Append_management(ruta_csv_relacion_municipio_capa, r_municipio_capa, "NO_TEST")
print(" - Se incorporan los registros de la relación: {0}".format(r_municipio_capa))

# ********************** Relación Municipio - Cuenca **********************
# - Creación de la estrucutura de la relación
r_municipio_cuenca = "r_municipio_cuenca"
arcpy.management.CreateRelationshipClass(tabla_cuenca, feature_municipio, r_municipio_cuenca, 'SIMPLE', 'cuenca', 'municipio', 
                                         "NONE", 'MANY_TO_MANY', "NONE", 'codigo_cuenca', 'codigo_cuenca', 'codigo_municipio', 'codigo_municipio')
print("Creada relación: {0}".format(r_municipio_cuenca))

# - Creación de la vista de tabla
ruta_csv_relacion_municipio_cuenca = r"D:\PUBLIC\PGAR\Resultados\2.Py\r_municipio_cuenca.csv"

# - Unificación de registros
arcpy.Append_management(ruta_csv_relacion_municipio_cuenca, r_municipio_cuenca, "NO_TEST")
print(" - Se incorporan los registros de la relación: {0}".format(r_municipio_cuenca))

<h4>Creación de relaciones N:1</h4>

In [ ]:
tabla_tematica = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.gdb\tbl_tematica"
tabla_direccion = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.gdb\direccion"
tabla_departamento = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.gdb\tbl_departamento"
tabla_fuente = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.gdb\tbl_fuentes"
r_capa_tematica = "r_capa_tematica"
r_direccion_municipio = "r_direccion_municipio"
r_municipio_departamento = "r_municipio_departamento"
r_capa_fuentes = "r_capa_fuentes"

# ********************** Relación Capa - Temática **********************
arcpy.management.CreateRelationshipClass(tabla_tematica, tabla_capa, r_capa_tematica, 'SIMPLE', 'capa', 'tematica', 
                                         "NONE", 'ONE_TO_MANY', "NONE", 'codigo_tematica', 'codigo_tematica')
print("Creada relación: {0}".format(r_capa_tematica))

# ********************** Relación Dirección - Municipio **********************
arcpy.management.CreateRelationshipClass(tabla_direccion, feature_municipio, r_direccion_municipio, 'SIMPLE', 'municipio', 'direccion', 
                                         "NONE", 'ONE_TO_MANY', "NONE", 'codigo_direccion', 'codigo_direccion')
print("Creada relación: {0}".format(r_direccion_municipio))

# ********************** Relación Municipio - Departamento **********************
arcpy.management.CreateRelationshipClass(tabla_departamento, feature_municipio, r_municipio_departamento, 'SIMPLE', 'municipio', 'departamento', 
                                         "NONE", 'ONE_TO_MANY', "NONE", 'codigo_departamento', 'codigo_departamento')
print("Creada relación: {0}".format(r_municipio_departamento))

# ********************** Relación Capa - Fuentes **********************
arcpy.management.CreateRelationshipClass(tabla_fuente, tabla_capa, r_capa_fuentes, 'SIMPLE', 'capa', 'fuente', 
                                         "NONE", 'ONE_TO_MANY', "NONE", 'codigo_fuente', 'codigo_fuente')
print("Creada relación: {0}".format(r_capa_fuentes))

<h4>Relación masiva entre la -tbl_capa- y cada una de las capas</h4>

In [ ]:
tabla_capa = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.gdb\tbl_capa"

# Proceso realizado en el archivo _9_...
for datasets in arcpy.ListDatasets():
    for capas in arcpy.ListFeatureClasses(feature_dataset=datasets):
            arcpy.management.AlterField(capas, 'codigo_capa', 'codigo_capa_borrar')
            arcpy.management.AddField(capas, 'codigo_capa', 'DOUBLE', field_alias = 'Código Capa')
            arcpy.management.CalculateField(capas, 'codigo_capa', '!codigo_capa_borrar!')
            print("-Código Capa- actualizado para: {0}".format(capas))
            
for datasets in arcpy.ListDatasets():
    for capas in arcpy.ListFeatureClasses(feature_dataset=datasets):        
        ruta_capa = os.path.join(arcpy.env.workspace,datasets,capas)
        if capas != 'municipio' or capas != 'cuenca':
            nombre_relacion = "r_capa_" + capas
            arcpy.management.CreateRelationshipClass(tabla_capa, ruta_capa, nombre_relacion, 'SIMPLE', 'registros', 'capa', 
                                                "NONE", 'ONE_TO_MANY', "NONE", 'codigo_capa', 'codigo_capa')
            print("Creada relación: {0}".format(nombre_relacion))

<h1>RELACIONES SOBRE LA EMPRESARIAL</h1>

<h4>Creación de relaciones M:N</h4>

In [ ]:
arcpy.env.workspace = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.sde"

feature_municipio = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.sde\municipio"
tabla_capa = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.sde\tbl_capa"
tabla_cuenca = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.sde\cuenca"

# ********************** Relación Municipio - Capa **********************
# - Creación de la estrucutura de la relación
r_municipio_capa = "r_municipio_capa"
arcpy.management.CreateRelationshipClass(tabla_capa, feature_municipio, r_municipio_capa, 'SIMPLE', 'capa', 'municipio', 
                                         "NONE", 'MANY_TO_MANY', "NONE", 'codigo_capa', 'codigo_capa', 'codigo_municipio', 'codigo_municipio')
print("Creada relación: {0}".format(r_municipio_capa))

# - Creación de la vista de tabla
ruta_csv_relacion_municipio_capa = r"D:\PUBLIC\PGAR\Resultados\2.Py\r_municipio_capa.csv"

# - Unificación de registros
arcpy.Append_management(ruta_csv_relacion_municipio_capa, r_municipio_capa, "NO_TEST")
print(" - Se incorporan los registros de la relación: {0}".format(r_municipio_capa))

# ********************** Relación Municipio - Cuenca **********************
# - Creación de la estrucutura de la relación
r_municipio_cuenca = "r_municipio_cuenca"
arcpy.management.CreateRelationshipClass(tabla_cuenca, feature_municipio, r_municipio_cuenca, 'SIMPLE', 'cuenca', 'municipio', 
                                         "NONE", 'MANY_TO_MANY', "NONE", 'codigo_cuenca', 'codigo_cuenca', 'codigo_municipio', 'codigo_municipio')
print("Creada relación: {0}".format(r_municipio_cuenca))

# - Creación de la vista de tabla
ruta_csv_relacion_municipio_cuenca = r"D:\PUBLIC\PGAR\Resultados\2.Py\r_municipio_cuenca.csv"

# - Unificación de registros
arcpy.Append_management(ruta_csv_relacion_municipio_cuenca, r_municipio_cuenca, "NO_TEST")
print(" - Se incorporan los registros de la relación: {0}".format(r_municipio_cuenca))

<h4>Creación de relaciones N:1</h4>

In [ ]:
tabla_tematica = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.sde\tbl_tematica"
tabla_direccion = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.sde\direccion"
tabla_departamento = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.sde\tbl_departamento"
tabla_fuente = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.sde\tbl_fuentes"
r_capa_tematica = "r_capa_tematica"
r_direccion_municipio = "r_direccion_municipio"
r_municipio_departamento = "r_municipio_departamento"
r_capa_fuentes = "r_capa_fuentes"

# ********************** Relación Capa - Temática **********************
arcpy.management.CreateRelationshipClass(tabla_tematica, tabla_capa, r_capa_tematica, 'SIMPLE', 'capa', 'tematica', 
                                         "NONE", 'ONE_TO_MANY', "NONE", 'codigo_tematica', 'codigo_tematica')
print("Creada relación: {0}".format(r_capa_tematica))

# ********************** Relación Dirección - Municipio **********************
arcpy.management.CreateRelationshipClass(tabla_direccion, feature_municipio, r_direccion_municipio, 'SIMPLE', 'municipio', 'direccion', 
                                         "NONE", 'ONE_TO_MANY', "NONE", 'codigo_direccion', 'codigo_direccion')
print("Creada relación: {0}".format(r_direccion_municipio))

# ********************** Relación Municipio - Departamento **********************
arcpy.management.CreateRelationshipClass(tabla_departamento, feature_municipio, r_municipio_departamento, 'SIMPLE', 'municipio', 'departamento', 
                                         "NONE", 'ONE_TO_MANY', "NONE", 'codigo_departamento', 'codigo_departamento')
print("Creada relación: {0}".format(r_municipio_departamento))

# ********************** Relación Capa - Fuentes **********************
arcpy.management.CreateRelationshipClass(tabla_fuente, tabla_capa, r_capa_fuentes, 'SIMPLE', 'capa', 'fuente', 
                                         "NONE", 'ONE_TO_MANY', "NONE", 'codigo_fuente', 'codigo_fuente')
print("Creada relación: {0}".format(r_capa_fuentes))

<h4>Relación masiva entre la -tbl_capa- y cada una de las capas</h4>

In [ ]:
arcpy.env.workspace = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.sde"
tabla_capa = r"D:\PUBLIC\PGAR\Resultados\4.PRO\PGAR\PGAR_Inventario.sde\tbl_capa"

# Proceso realizado en el archivo _9_...
for datasets in arcpy.ListDatasets():
    for capas in arcpy.ListFeatureClasses(feature_dataset=datasets):
            arcpy.management.AlterField(capas, 'codigo_capa', 'codigo_capa_borrar')
            arcpy.management.AddField(capas, 'codigo_capa', 'DOUBLE', field_alias = 'Código Capa')
            arcpy.management.CalculateField(capas, 'codigo_capa', '!codigo_capa_borrar!')
            print("-Código Capa- actualizado para: {0}".format(capas))
            
for datasets in arcpy.ListDatasets():
    for capas in arcpy.ListFeatureClasses(feature_dataset=datasets):        
        ruta_capa = os.path.join(arcpy.env.workspace,datasets,capas)
        if capas != 'municipio' or capas != 'cuenca':
            nombre_relacion = "r_capa_" + capas
            arcpy.management.CreateRelationshipClass(tabla_capa, ruta_capa, nombre_relacion, 'SIMPLE', 'registros', 'capa', 
                                                "NONE", 'ONE_TO_MANY', "NONE", 'codigo_capa', 'codigo_capa')
            print("Creada relación: {0}".format(nombre_relacion))